In [1]:
# # 데이터 과학 기반의 파이썬 빅데이터 분석 

In [2]:
# asdfzzz / 1q???

#my_key = "s1JerABCGMtqqLUB0r7byT8n7uzwvD9lPsxSfKp9ID5seIFNMkOQzIsACpahzhcUznSQXlirur4Ivp38W2Zd3Q%3D%3D"  # 2019/10/25

In [1]:
my_key = "0C6iX2Ww%2BnB9GVY6k9Et5H9zqv%2FxjEsldVq1Wa9Co3lvcuaRi%2BI3gcoFoi3AckHg6B8AAf1cH0tKH4MXhYf57A%3D%3D" # 2020/08/11

## ( 공공데이터 포털 API 사용 )
1. 공공데이터 포털 가입(data.go.kr)
   - 로그인 
   - 한국문화관광연구원_)출입국관광통계서비스 
   - 활용신청
   - 시스템유형(일반), 활용정보(웹사이트 개발)
   - 상세기능정보(선택)
   - 동의 합니다. - 신청
2. OpenAPI 활용 가이드.hwp
3. 인증키 발급현황
   - 인증키 복사 
2. 개발계정 인반 인증키 - 복사
   - End point
   - 데이터 포맷
   - 참고문서
3. URL 만들기 
   - end_point + operation + key + data_format
   - url 확인 (브라우저)

In [1]:
my_key = "0C6iX2Ww%2BnB9GVY6k9Et5H9zqv%2FxjEsldVq1Wa9Co3lvcuaRi%2BI3gcoFoi3AckHg6B8AAf1cH0tKH4MXhYf57A%3D%3D"

In [2]:
#http://apis.data.go.kr/B553077/api/open/sdsc/smallUpjongList?serviceKey=0C6iX2Ww%2BnB9GVY6k9Et5H9zqv%2FxjEsldVq1Wa9Co3lvcuaRi%2BI3gcoFoi3AckHg6B8AAf1cH0tKH4MXhYf57A%3D%3D&indsLclsCd=Q&indsMclsCd=Q01

In [13]:
import urllib.request
import json

def get_visit(url):
    req = urllib.request.Request(url)
    response = urllib.request.urlopen(req)
    res_read = response.read().decode("utf-8)")
    json_load = json.loads(res_read)
    visit_ym  = json_load['response']['body']['items']['item']['ym']    # 년월
    visit_country = json_load['response']['body']['items']['item']['natKorNm'] # 
    visit_count = json_load['response']['body']['items']['item']['num'] # 방문객수
    visit_class = json_load['response']['body']['items']['item']['ed']  # 구분(방한외래관광객)
    
    return [visit_ym, visit_country, visit_count, visit_class ]

In [14]:
country_code = ["112","130","275"]  # 국가코드 중국(112), 일본(130), 미국(275)
visit_year = [2019,2020] # 2019년01월 ~ 2020년 12월 
ed_cd = "E"  # 방한외래관광객(E), 해외출국(D) 

service_url = "http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList"
url_01      = "?_type=json&serviceKey=" + my_key   #인증키

total_visit = []

for country in country_code:
    for year in visit_year:
        for month in range(1,13):
            url_02 = "&YM=" + str(year)+"%02d"%month
            url_02 += "&NAT_CD=" + country
            url_02 += "&ED_CD=" + ed_cd            
            url = service_url + url_01 + url_02

            result = get_visit(url)             
            total_visit.append(result)
            
import pandas as pd

df = pd.DataFrame(total_visit, columns = ["년월","국가","방문객수","구분"])
df.shape

(72, 4)

In [5]:
df

,년월,국가,방문객수,구분
0,201901,중 국,392814,방한외래관광객
1,201902,중 국,453379,방한외래관광객
2,201903,중 국,487623,방한외래관광객
3,201904,중 국,493250,방한외래관광객
4,201905,중 국,500413,방한외래관광객
...,...,...,...,...
67,202008,미 국,13368,방한외래관광객
68,202009,미 국,12426,방한외래관광객
69,202010,미 국,12366,방한외래관광객
70,202011,미 국,13100,방한외래관광객


## ( 2. 딕셔너리 처리 )

In [15]:
def get_visit_01(url):
    req = urllib.request.Request(url)
    response = urllib.request.urlopen(req)
    res_read = response.read().decode("utf-8)")
    json_load = json.loads(res_read)
    item  = json_load['response']['body']['items']['item']         
    return item

In [16]:
country_code = ["112","130","275"]  # 국가코드 중국(112), 일본(130), 미국(275)
visit_year = [2019,2020] # 2019년01월 ~ 2020년 12월 
ed_cd = "E"  # 방한외래관광객(E), 해외출국(D) 

service_url = "http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList"
url_01      = "?_type=json&serviceKey=" + my_key   #인증키

total_visit = []
init = 0 
for country in country_code:
    for year in visit_year:
        for month in range(1,13):
            url_02 = "&YM=" + str(year)+"%02d"%month
            url_02 += "&NAT_CD=" + country
            url_02 += "&ED_CD=" + ed_cd            
            url = service_url + url_01 + url_02

            result = get_visit_01(url) 
            
            df = pd.DataFrame([result])
            
            if init == 0:
                df_total = df
                init = 1
            else:
                df_total = pd.concat([df_total, df]) 
                
df_total.shape 

df_total = df_total[["ym","natKorNm","num","ed"]]
df_total.columns = ["년월","국가","방문객수","구분"]
df_total.head() 

,년월,국가,방문객수,구분
0,201901,중 국,392814,방한외래관광객
0,201902,중 국,453379,방한외래관광객
0,201903,중 국,487623,방한외래관광객
0,201904,중 국,493250,방한외래관광객
0,201905,중 국,500413,방한외래관광객


In [6]:
df_total.shape

(72, 4)

## ( 3. Parameter  처리 )

In [17]:
import requests
def get_visit_02(url, params):
    req = requests.get(url, params) 
    print("req = ", req.json())
    item  = req.json()['response']['body']['items']['item']    
    return item

In [18]:
country_code = ["112","130","275"]  # 국가코드 중국(112), 일본(130), 미국(275)
visit_year = [2019,2020] # 2019년01월 ~ 2020년 12월 
ed_cd = "E"  # 방한외래관광객(E), 해외출국(D) 

service_url = "http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList"
url_01      = "?_type=json&serviceKey=" + my_key   #인증키

params = {"YM": str(year)+"%02d"%month,
        "NAT_CD":country,
        "ED_CD":ed_cd}

total_visit = []
init = 0 
for country in country_code:
    for year in visit_year:
        for month in range(1,13):
            
            params = {"YM": str(year)+"%02d"%month,
                      "NAT_CD":country,
                      "ED_CD":ed_cd} 
            
            print("params = ", params)
            
            result = get_visit_02(url, params=params) 
            
            print("result = ", result)

params =  {'YM': '201901', 'NAT_CD': '112', 'ED_CD': 'E'}
req =  {'response': {'header': {'resultCode': '0000', 'resultMsg': 'OK'}, 'body': {'items': {'item': {'ed': '방한외래관광객', 'edCd': 'E', 'natCd': 275, 'natKorNm': '미  국', 'num': 12102, 'rnum': 1, 'ym': 202012}}, 'numOfRows': 10, 'pageNo': 1, 'totalCount': 1}}}
result =  {'ed': '방한외래관광객', 'edCd': 'E', 'natCd': 275, 'natKorNm': '미  국', 'num': 12102, 'rnum': 1, 'ym': 202012}
params =  {'YM': '201902', 'NAT_CD': '112', 'ED_CD': 'E'}
req =  {'response': {'header': {'resultCode': '0000', 'resultMsg': 'OK'}, 'body': {'items': {'item': {'ed': '방한외래관광객', 'edCd': 'E', 'natCd': 275, 'natKorNm': '미  국', 'num': 12102, 'rnum': 1, 'ym': 202012}}, 'numOfRows': 10, 'pageNo': 1, 'totalCount': 1}}}
result =  {'ed': '방한외래관광객', 'edCd': 'E', 'natCd': 275, 'natKorNm': '미  국', 'num': 12102, 'rnum': 1, 'ym': 202012}
params =  {'YM': '201903', 'NAT_CD': '112', 'ED_CD': 'E'}
req =  {'response': {'header': {'resultCode': '0000', 'resultMsg': 'OK'}, 'body': {'it

req =  {'response': {'header': {'resultCode': '0000', 'resultMsg': 'OK'}, 'body': {'items': {'item': {'ed': '방한외래관광객', 'edCd': 'E', 'natCd': 275, 'natKorNm': '미  국', 'num': 12102, 'rnum': 1, 'ym': 202012}}, 'numOfRows': 10, 'pageNo': 1, 'totalCount': 1}}}
result =  {'ed': '방한외래관광객', 'edCd': 'E', 'natCd': 275, 'natKorNm': '미  국', 'num': 12102, 'rnum': 1, 'ym': 202012}
params =  {'YM': '201904', 'NAT_CD': '130', 'ED_CD': 'E'}
req =  {'response': {'header': {'resultCode': '0000', 'resultMsg': 'OK'}, 'body': {'items': {'item': {'ed': '방한외래관광객', 'edCd': 'E', 'natCd': 275, 'natKorNm': '미  국', 'num': 12102, 'rnum': 1, 'ym': 202012}}, 'numOfRows': 10, 'pageNo': 1, 'totalCount': 1}}}
result =  {'ed': '방한외래관광객', 'edCd': 'E', 'natCd': 275, 'natKorNm': '미  국', 'num': 12102, 'rnum': 1, 'ym': 202012}
params =  {'YM': '201905', 'NAT_CD': '130', 'ED_CD': 'E'}
req =  {'response': {'header': {'resultCode': '0000', 'resultMsg': 'OK'}, 'body': {'items': {'item': {'ed': '방한외래관광객', 'edCd': 'E', 'natCd': 275

req =  {'response': {'header': {'resultCode': '0000', 'resultMsg': 'OK'}, 'body': {'items': {'item': {'ed': '방한외래관광객', 'edCd': 'E', 'natCd': 275, 'natKorNm': '미  국', 'num': 12102, 'rnum': 1, 'ym': 202012}}, 'numOfRows': 10, 'pageNo': 1, 'totalCount': 1}}}
result =  {'ed': '방한외래관광객', 'edCd': 'E', 'natCd': 275, 'natKorNm': '미  국', 'num': 12102, 'rnum': 1, 'ym': 202012}
params =  {'YM': '201907', 'NAT_CD': '275', 'ED_CD': 'E'}
req =  {'response': {'header': {'resultCode': '0000', 'resultMsg': 'OK'}, 'body': {'items': {'item': {'ed': '방한외래관광객', 'edCd': 'E', 'natCd': 275, 'natKorNm': '미  국', 'num': 12102, 'rnum': 1, 'ym': 202012}}, 'numOfRows': 10, 'pageNo': 1, 'totalCount': 1}}}
result =  {'ed': '방한외래관광객', 'edCd': 'E', 'natCd': 275, 'natKorNm': '미  국', 'num': 12102, 'rnum': 1, 'ym': 202012}
params =  {'YM': '201908', 'NAT_CD': '275', 'ED_CD': 'E'}
req =  {'response': {'header': {'resultCode': '0000', 'resultMsg': 'OK'}, 'body': {'items': {'item': {'ed': '방한외래관광객', 'edCd': 'E', 'natCd': 275

In [29]:
import requests 

def get_visit03(url):
    req = requests.get(url) 
    json_loads = json.loads(req.text)
    df = pd.DataFrame(json_loads["response"]["body"]["items"]).T    
    return df

In [42]:
country_code = ["112","130","275"]  # 국가코드 중국(112), 일본(130), 미국(275)
visit_year = [2019,2020] # 2019년01월 ~ 2020년 12월 
ed_cd = "E"  # 방한외래관광객(E), 해외출국(D) 

service_url = "http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList"
url_01      = "?_type=json&serviceKey=" + my_key   #인증키

total_visit = []

init = 0

for country in country_code:
    for year in visit_year:
        for month in range(1,13):
            url_02 = "&YM=" + str(year)+"%02d"%month
            url_02 += "&NAT_CD=" + country
            url_02 += "&ED_CD=" + ed_cd            
            url = service_url + url_01 + url_02

            visit_page = get_visit03(url)            

            if init == 0:
                visit_total  = visit_page.copy()
                init = 1
            else:
                visit_total = pd.concat([visit_total, visit_page])
                
visit_total = visit_total[["ym","natKorNm","num","edCd"]] 
visit_total.columns = ["년월","국가","방문객수","구분"] 

visit_total.reset_index().drop("index", axis=1)

,년월,국가,방문객수,구분
0,201901,중 국,392814,E
1,201902,중 국,453379,E
2,201903,중 국,487623,E
3,201904,중 국,493250,E
4,201905,중 국,500413,E
...,...,...,...,...
67,202008,미 국,13368,E
68,202009,미 국,12426,E
69,202010,미 국,12366,E
70,202011,미 국,13100,E
